# Sesión 2

In [83]:
import nltk
# Punkt permite separar un texto en frases.
nltk.download('punkt')
# Descarga todas las palabras vacias, es decir, aquellas que no aportan nada al significado del texto
nltk.download('stopwords')
#Paquete WordNetLemmatizer, este es usado para encontrar el lema de cada palabr
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marcosrodrigo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marcosrodrigo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/marcosrodrigo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [84]:
# Librerias
import pandas as pd
import numpy as np
import sys
!{sys.executable} -m pip install pandas-profiling

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from datetime import datetime


from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

1. Perfilamiento, entendimiento de los datos y  análisis de la calidad de los datos

Carga de datos

In [85]:
data_cargada=pd.read_csv('Data/fake_news_spanish.csv', sep=';', encoding = 'utf-8')
# Asignación a una nueva variable de los datos leidos
df=data_cargada

In [86]:

df.head(10)

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018
5,ID,1,"El PNV consolida su mayoría, el PSE salva los ...",Los nacionalistas consiguen las alcaldías de B...,26/05/2019
6,ID,0,El exconsejero Núria Marín pide el indulto en ...,Sus familiares aluden a su honestidad e integr...,16/09/2022
7,ID,1,La Fiscalía pide prisión incondicional para lo...,Suprime el delito de rebelión que les imputó i...,26/09/2019
8,ID,1,"José Manuel Pérez Tornero, el creador de la te...",El futuro presidente de RTVE es licenciado en ...,25/02/2021
9,ID,0,La 'Ayusización' del BNG: Santiago Abascal ins...,Pablo Santiago Abascal planea vivir de las ren...,10/05/2021


In [87]:
df.dtypes

ID             object
Label           int64
Titulo         object
Descripcion    object
Fecha          object
dtype: object

Debemos de hacer una correccion de los datos
Titulo: string (se puede dejar como object, pero pandas tiene el tipo string que es más eficiente)
Descripcion: string (igual que el título)
Fecha: datetime64 (definitivamente no debería ser object)

In [88]:
# Convertir ID a numérico
# data['ID'] = pd.to_numeric(data['ID'])

# Convertir columnas de texto a string
df['Titulo'] = df['Titulo'].astype('string')
df['Descripcion'] = df['Descripcion'].astype('string')

# Convertir Fecha a datetime
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')

# Entendimiento de los datos

# Limpieza de los datos

In [89]:
df.isnull().sum()


ID              0
Label           0
Titulo         16
Descripcion     0
Fecha           0
dtype: int64

In [90]:
df[df.isna().any(axis=1)]


,ID,Label,Titulo,Descripcion,Fecha
3243,ID,0,<NA>,"Hace unos 75 años, Hermann Göring testificó en...",2019-06-16
4189,ID,0,<NA>,"Evidentemente, Barak Obama ha sido arrestado e...",2022-03-10
5041,ID,0,<NA>,"""Después de convertirme en presidente, le pedí...",2022-03-16
5644,ID,0,<NA>,Me pasaron de buena fuente hoy (un vecino del ...,2023-04-08
7345,ID,0,<NA>,Defienden dichos con capitulos de Don Gato. C...,2023-07-02
7652,ID,0,<NA>,Lean (y vean la imagen) con mucha atención: S...,2018-08-04
9700,ID,0,<NA>,¿Sabías que Francisco Sagasti salió a marchar ...,2018-12-26
15159,ID,0,<NA>,Declarar a Bill Gates enemigo público y proces...,2024-08-02
19415,ID,0,<NA>,Con el fin de captar votos están dispuestos a ...,2017-10-28
24806,ID,0,<NA>,CUANDO LAS IMÁGENES HABLAN MÁS QUE PALABRAS. ...,2018-06-28


In [91]:
df = df.dropna()
#Elimine los datos que no tienen titulo REVISAR

In [92]:
palabras_sospechosas = ['secreto', 'impactante', 'milagrosa', 'urgente']
# Palabras que nos pueden llegar a dirgir que un texto es falso

In [93]:
def extraer_caracteristicas_texto(texto, tipo='titulo'):
    tokens = word_tokenize(texto.lower())
    num_palabras = len(tokens)
    num_sospechosas = sum(1 for palabra in tokens if palabra in palabras_sospechosas)
    # Simulamos un puntaje de sentimiento (0 a 1, más alto = más emocional)
    sentimiento = 0.8 if num_sospechosas > 0 else 0.3  # Ejemplo simple
    if tipo == 'titulo':
        return num_palabras, num_sospechosas, sentimiento
    return num_sospechosas, sentimiento


# Crear columnas con las características
df[['Palabras_Título', 'Sospechosas_Título', 'Sentimiento_Título']] = df['Titulo'].apply(
    lambda x: pd.Series(extraer_caracteristicas_texto(x, 'titulo'))
)
df[['Sospechosas_Descripción', 'Sentimiento_Descripción']] = df['Descripcion'].apply(
    lambda x: pd.Series(extraer_caracteristicas_texto(x, 'descripcion'))
)

# Seleccionar las características para el modelo
X = df[['Palabras_Título', 'Sospechosas_Título', 'Sentimiento_Título',
        'Sospechosas_Descripción', 'Sentimiento_Descripción','Fecha']]
y = df['Label']

/var/folders/1j/z75cknhx76s2v56sd6b61_5w0000gn/T/ipykernel_14583/1369783005.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Palabras_Título', 'Sospechosas_Título', 'Sentimiento_Título']] = df['Titulo'].apply(
/var/folders/1j/z75cknhx76s2v56sd6b61_5w0000gn/T/ipykernel_14583/1369783005.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Palabras_Título', 'Sospechosas_Título', 'Sentimiento_Título']] = df['Titulo'].apply(
/var/folders/1j/z75cknhx76s2v56sd6b61_5w0000gn/T/ipykernel_14583/136978300

In [94]:
df

,ID,Label,Titulo,Descripcion,Fecha,Palabras_Título,Sospechosas_Título,Sentimiento_Título,Sospechosas_Descripción,Sentimiento_Descripción
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,2023-06-02,16.0,0.0,0.3,0.0,0.3
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,2023-10-01,18.0,0.0,0.3,0.0,0.3
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,2022-04-25,15.0,0.0,0.3,0.0,0.3
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",2022-01-03,20.0,0.0,0.3,0.0,0.3
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",2018-03-09,12.0,0.0,0.3,0.0,0.3
...,...,...,...,...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,2021-06-08,20.0,0.0,0.3,0.0,0.3
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,2020-09-08,20.0,0.0,0.3,0.0,0.3
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,2018-07-12,15.0,0.0,0.3,0.0,0.3
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,2022-02-13,22.0,0.0,0.3,0.0,0.3


# Regresion Logistica

Generamos el set de datos para la regresion logistica

In [95]:
dataRegresionLogistica=df

In [96]:
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train, y_train)

NameError: name 'X_train' is not defined